<a href="https://colab.research.google.com/github/leedain0301/Data_MiniProject_02/blob/main/MiniProject_02(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 컬럼 설명

#   train.csv / test.csv <br>
>  : 서울 지역의 740,000여개 거래 데이터, 아파트 거래일, 지역, 전용면적, 실 거래가 등의 정보

- **features**
  *   transaction_id : 아파트 거래 유일키키
  *   apartment_id : 아파트 아이디
  *   city : 도시
  *   dong : 동
  *   jibun : 지번
  *   apt : 아파트단지 이름
  *   addr_kr : 주소
  *   exclusive_use_area : 전용면적
  *   year_of_completion : 설립일자
  *   transaction_year_month : 거래년월
  *   transaction_date : 거래날짜
  *   floor : 층
  *   transaction_real_price : 실거래가(train 만 존재)


#   park.csv<br>
>  : 서울 지역의 공원에 대한 정보

- **features**

  *  city : 도시 이름
  *  gu : 구
  *  dong : 동
  *  park_name : 공원 이름
  *  park_type : 공원 이름
  *  park_area : 공원의 넓이
  *  park_exercise_facility : 공원보유 운동시설
  *  park_entertainment_facility : 공원보유 유희시설
  *  park_benefit_facility : 공원보유 편익시설
  *  park_cultural_facitiy : 공원보유 교양시설
  *  park_facility_other : 공원보유 기타시설
  *  park_open_year : 공원 개장년도
  *  reference_date : 데이터 기준일자(해당 데이터가 기록된 일자)



<br>

---

<br>









In [2]:
!pip install category_encoders
!pip3 install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import sklearn
from sklearn.linear_model    import ElasticNet, Lasso
from sklearn.ensemble        import GradientBoostingRegressor, RandomForestRegressor
from sklearn.kernel_ridge    import KernelRidge
from sklearn.pipeline        import make_pipeline
from sklearn.preprocessing   import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics         import mean_squared_error, accuracy_score
from sklearn.preprocessing   import StandardScaler
from sklearn.impute          import SimpleImputer
from category_encoders       import OrdinalEncoder
from category_encoders       import OneHotEncoder, OrdinalEncoder

import xgboost as XGBClassifier
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import sys

In [4]:
train = pd.read_csv('/content/drive/MyDrive/02.Colab Notebooks/MiniProject_02/tarin_clean.csv')
test = pd.read_csv('/content/drive/MyDrive/02.Colab Notebooks/MiniProject_02/test_clean.csv')
submission_data = pd.read_csv('/content/drive/MyDrive/02.Colab Notebooks/MiniProject_02/submission.csv')

print('train.csv. Shape: ',train.shape)
print('test.csv. Shape: ', test.shape)

train.csv. Shape:  (593828, 9)
test.csv. Shape:  (3911, 8)


In [5]:
train = train.drop('Unnamed: 0', axis=1)
test = test.drop('Unnamed: 0', axis=1)

train.head()

,dong,exclusive_use_area,year_of_completion,transaction_date,floor,transaction_real_price,year,month
0,상계동,59.95,1998,11~20,2.944439,10.221978,2008,3
1,쌍문동,84.53,1987,21~30,2.772589,10.288784,2009,9
2,창동,58.01,1990,1~10,2.833213,10.165890,2008,5
3,둔촌동,58.33,2004,11~20,2.639057,10.034560,2009,1
4,구로동,49.94,1987,21~31,2.197225,10.146473,2015,8


In [6]:
baseline_score = train['transaction_real_price'].value_counts(normalize=True).max()
print("baseline score:", baseline_score)

baseline score: 0.011016658022188243


### **< 데이터셋 분리 >**

---


In [7]:
train, val = train_test_split(
    train, train_size=0.80, test_size=0.20,  random_state=42
)

train.shape, val.shape, test.shape

((475062, 8), (118766, 8), (3911, 7))

In [8]:
target = 'transaction_real_price'
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

### **< Data processing >**

---


In [9]:
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier

baseline = [y_train.mean()] * len(y_train)
baseline_r2 = r2_score(y_train, baseline)
baseline_mae = mean_absolute_error(y_train, baseline)
print(f'기준모델의 r2_score: {baseline_r2}')
print(f'기준모델의 mae : {baseline_mae}')

기준모델의 r2_score: 0.0
기준모델의 mae : 0.42856806483177207


In [10]:
encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train) 
X_val_encoded = encoder.transform(X_val) 

boosting = XGBClassifier(
    n_estimators=500,
    objective='reg:logistic',  # default
    # learning_rate=0.2,
    # subsample=0.8,
    # colsample_bytree=0.8,
)
print(boosting)

XGBClassifier(n_estimators=500, objective='reg:logistic')


In [ ]:
boosting.fit(X_train_encoded, y_train)

In [ ]:
y_pred_proba = boosting.predict_proba(
    X_val_encoded, iteration_range=(0, boosting.best_iteration)
)[:, 1]
y_pred = boosting.predict(X_val_encoded, iteration_range=(0, boosting.best_iteration))
print("검증 accuracy", boosting.score(X_val_encoded, y_val))
print("검증 AUC", roc_auc_score(y_val, y_pred_proba))
print(classification_report(y_val, y_pred))

In [ ]:
k = train.shape[1] 
corrmat = train.corr() 
cols = corrmat.nlargest(k, 'transaction_real_price')['transaction_real_price'].index 
cm = np.corrcoef(train[cols].values.T)
f, ax = plt.subplots(figsize = (20, 6))
sns.heatmap(data = cm, 
            annot=True, 
            square=True, 
            fmt = '.2f', 
            linewidths=.5, 
            cmap='Reds', 
            yticklabels = cols.values, 
            xticklabels = cols.values
            )